# Prepare

In [ ]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import os

In [ ]:
from sklearn.metrics import confusion_matrix

def print_confusion_matrix(network, test_loader, num_classes:int):
  predictions = []
  labels = []
  network.eval()
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      predictions += list(output.argmax(1).data)
      labels += list(target.data)
  print('confusion matrix: ')
  predictions = [int(x) for x in predictions]
  labels = [int(x) for x in labels]
  print(confusion_matrix(labels, predictions))

In [ ]:
class NegativePenaltySparseCategoricalCrossentropy(nn.Module):
  def __init__(self, class_num:int, p_indices:list, alpha=1.0, penalty_scale=None, reduction='mean', \
         from_where='softmax', eps=1e-10, name='negative_penalty_sparse_categorical_crossentropy'):
    super(NegativePenaltySparseCategoricalCrossentropy, self).__init__()
    self.p_indices = [[p_index] for p_index in p_indices]
    self.alpha = alpha
    self.penalty_scale = float(len(p_indices)) if penalty_scale is None else penalty_scale
    self.penalty_label = _get_penalty_label(class_num, p_indices)
    self.reduction_fn = {
        'none': _no_reduction_over_batch, 'mean': _average_over_batch,
        'sum': _sum_over_batch
    }[reduction]
    self.cce_loss_fn = {
        'logits': _cce_loss_from_logits, 'softmax': _cce_loss_from_softmax,
    }[from_where]
    self.penalty_loss_fn = {
        'logits': _penalty_loss_from_logits, 'softmax': _penalty_loss_from_softmax,
    }[from_where]
    self.eps = eps

  def forward(self, y_pred, y_true):
    num_classes = y_pred.shape[-1]
    # y_true = torch.squeeze(F.one_hot(y_true, num_classes=num_classes), dim=1)
    y_true = F.one_hot(y_true, num_classes=num_classes).float()
    losses = _get_losses(
        y_true, y_pred, self.p_indices, self.penalty_label, self.alpha, self.penalty_scale, self.eps,
        self.cce_loss_fn, self.penalty_loss_fn
    )
    losses = self.reduction_fn(losses)
    return losses


def _get_losses(y_true, y_pred, p_indices:list, penalty_label:list, alpha:float, penalty_scale:float,
         eps:float, cce_loss_fn, penalty_loss_fn):
  batch_size = y_true.shape[0]
  # cce_loss_sample_weights
  cce_loss_sample_weights = torch.any(
      torch.transpose(torch.eq(torch.tensor(p_indices), torch.argmax(y_true, dim=-1)), 0, 1), dim=-1
  ).float()
  # cce loss
  cce_losses = cce_loss_fn(y_pred, y_true, eps)
  cce_losses = cce_loss_sample_weights * cce_losses
  # y_penalty
  y_penalty = torch.repeat_interleave(torch.unsqueeze(torch.tensor(penalty_label), dim=0), batch_size, dim=0).float()
  # penalty_loss_sample_weights
  penalty_loss_sample_weights = 1.0 - cce_loss_sample_weights
  # penalty loss
  penalty_losses = penalty_loss_fn(y_pred, y_penalty, penalty_scale, eps)
  penalty_losses = penalty_loss_sample_weights * penalty_losses
  # total loss
  losses = cce_losses + alpha * penalty_losses
  return losses


def _no_reduction_over_batch(losses):
  return losses


def _average_over_batch(losses):
  return torch.mean(losses)


def _sum_over_batch(losses):
  return torch.sum(losses)


def _cce_loss_from_logits(y_pred, y_true, eps):
  return F.cross_entropy(y_pred, y_true, reduction='none')


def _cce_loss_from_softmax(y_pred, y_true, eps):
  return torch.sum(-y_true * torch.log(torch.clip(y_pred, eps, 1.0 - eps)), dim=-1)


def _penalty_loss_from_logits(y_pred, y_penalty, penalty_scale, eps):
  return F.cross_entropy(1.0 - y_pred, y_penalty, reduction='none') / penalty_scale


def _penalty_loss_from_softmax(y_pred, y_penalty, penalty_scale, eps):
  return torch.sum(
      -y_penalty * torch.log(torch.clip(1.0 - y_pred, eps, 1.0 - eps)), dim=-1
  ) / penalty_scale


def _get_penalty_label(class_num:int, p_indices:list):
  penalty_label = [1 if i in p_indices else 0 for i in range(0, class_num)]
  return penalty_label

In [ ]:
class CustomCrossEntropyLoss(nn.Module):
  def __init__(self, eps=1e-10):
    super(CustomCrossEntropyLoss, self).__init__()
    self.eps = eps

  def forward(self, output, target):
    num_classes = output.shape[-1]
    target = F.one_hot(target, num_classes=num_classes).float()
    return torch.sum(-target * torch.log(torch.clip(output, self.eps, 1.0 - self.eps)), dim=-1).mean()

In [ ]:
def train(network, train_loader, optimizer, criterion, epoch, train_losses, train_counter):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 1000 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), 'results/model.pth')
      torch.save(optimizer.state_dict(), 'results/optimizer.pth')

def test(network, test_loader, criterion, test_losses):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print(
    '\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset))
  )

In [ ]:
# Define the model, loss function and optimizer
class Net(nn.Module):
  def __init__(self, from_where:str):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 14)
    self.from_where = from_where

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    if self.from_where == 'logits':
      return x
    elif self.from_where == 'softmax':
      return F.softmax(x, dim=-1)

In [7]:
# Load the MNIST dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(
    '/files/', train=True, download=True,
    transform=torchvision.transforms.Compose(
      [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
      ]
    )
  ),
  batch_size=32,
  shuffle=True
)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(
    '/files/', train=False, download=True,
    transform=torchvision.transforms.Compose(
      [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
      ]
    )
  ),
  batch_size=32,
  shuffle=True
)

# Simply train model on mnist dataset with normal categorical crossentroy loss

1. from_where = 'logits'

In [8]:
network = Net('logits')
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
# Training loop
n_epochs = 3
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
if os.path.exists('results'):
  os.system('rm -r results')
os.mkdir('results')
for epoch in range(1, n_epochs + 1):
  train(network, train_loader, optimizer, criterion, epoch, train_losses, train_counter)
  test(network, test_loader, criterion, test_losses)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.752748
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.394198

Test set: Avg. loss: 0.0026, Accuracy: 9742/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.163537
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.442460

Test set: Avg. loss: 0.0020, Accuracy: 9800/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.358956
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.064064

Test set: Avg. loss: 0.0016, Accuracy: 9839/10000 (98%)



plot confusion matrix

In [9]:
print_confusion_matrix(network, test_loader, num_classes=14)

confusion matrix: 
[[ 975    0    1    0    0    0    2    1    1    0]
 [   0 1126    6    0    0    1    1    0    1    0]
 [   2    1 1021    2    1    0    0    3    2    0]
 [   1    0    5  990    0    8    0    2    4    0]
 [   0    0    2    0  966    0    1    1    1   11]
 [   2    0    0    4    0  883    2    1    0    0]
 [   7    2    0    0    1    2  946    0    0    0]
 [   0    3   20    2    0    1    0  993    3    6]
 [   1    0    3    1    2    0    0    0  965    2]
 [   3    3    0    4    8    5    0    6    6  974]]


2. from_where == 'softmax'

In [10]:
network = Net('softmax')
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = CustomCrossEntropyLoss()
# Training loop
n_epochs = 3
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
if os.path.exists('results'):
  os.system('rm -r results')
os.mkdir('results')
for epoch in range(1, n_epochs + 1):
  train(network, train_loader, optimizer, criterion, epoch, train_losses, train_counter)
  test(network, test_loader, criterion, test_losses)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.617424
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.392073

Test set: Avg. loss: 0.0029, Accuracy: 9696/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.534394
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.200317

Test set: Avg. loss: 0.0021, Accuracy: 9789/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.231865
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.225195

Test set: Avg. loss: 0.0018, Accuracy: 9801/10000 (98%)



plot confusion matrix

In [11]:
print_confusion_matrix(network, test_loader, num_classes=14)

confusion matrix: 
[[ 973    0    2    0    0    0    4    1    0    0]
 [   0 1117    1    3    0    2    3    2    7    0]
 [   1    2 1000    6    2    0    2   17    2    0]
 [   0    0    2  992    0    8    0    6    1    1]
 [   0    0    1    0  962    0    2    1    0   16]
 [   3    0    0    6    0  879    3    1    0    0]
 [   6    2    1    0    1    4  944    0    0    0]
 [   0    1   10    3    0    0    0 1010    0    4]
 [   5    0    4    2    2    5    1    5  942    8]
 [   2    3    0    3    6    4    1    7    1  982]]


# Train model on mnist dataset with the proposed 'NegativePenaltySparseCategoricalCrossentropy' loss function

1. from_where = 'logits'

In [ ]:
network = Net('logits')
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = NegativePenaltySparseCategoricalCrossentropy(class_num=14, p_indices=[0, 1, 2, 3, 4, 5, 6, 7], from_where='logits')
# Training loop
n_epochs = 5
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
if os.path.exists('results'):
  os.system('rm -r results')
os.mkdir('results')
for epoch in range(1, n_epochs + 1):
  train(network, train_loader, optimizer, criterion, epoch, train_losses, train_counter)
  test(network, test_loader, criterion, test_losses)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.637293
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.652617

Test set: Avg. loss: 0.0156, Accuracy: 7901/10000 (79%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.400145
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.735097

Test set: Avg. loss: 0.0147, Accuracy: 8419/10000 (84%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.689269


plot confusion matrix

In [13]:
print_confusion_matrix(network, test_loader, num_classes=14)

confusion matrix: 
[[ 976    0    1    0    0    0    2    1    0    0    0    0    0    0]
 [   0 1127    2    0    0    1    4    0    0    0    0    1    0    0]
 [   2    1 1019    1    1    0    2    5    0    0    0    1    0    0]
 [   0    0    1  995    0    6    0    5    0    0    0    3    0    0]
 [   0    0    0    0  980    0    0    0    0    0    0    2    0    0]
 [   2    0    0    3    0  880    6    1    0    0    0    0    0    0]
 [   4    2    0    0    1    1  950    0    0    0    0    0    0    0]
 [   0    2   20    2    0    0    0 1001    0    0    0    3    0    0]
 [  18    0    5    7    6    8    3    3  443   11   35  427    8    0]
 [   7    3    0    4   11    6    0    6  520    2   53  392    4    1]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0 

2. from_where = 'softmax'

In [14]:
network = Net('softmax')
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = NegativePenaltySparseCategoricalCrossentropy(class_num=14, p_indices=[0, 1, 2, 3, 4, 5, 6, 7], from_where='softmax')
# Training loop
n_epochs = 5
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
if os.path.exists('results'):
  os.system('rm -r results')
os.mkdir('results')
for epoch in range(1, n_epochs + 1):
  train(network, train_loader, optimizer, criterion, epoch, train_losses, train_counter)
  test(network, test_loader, criterion, test_losses)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299968
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.388778

Test set: Avg. loss: 0.0027, Accuracy: 7838/10000 (78%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.102295
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.144349

Test set: Avg. loss: 0.0019, Accuracy: 7906/10000 (79%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.090366
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.045633

Test set: Avg. loss: 0.0015, Accuracy: 7917/10000 (79%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.052283
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.036018

Test set: Avg. loss: 0.0013, Accuracy: 7938/10000 (79%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.057037
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.270482

Test set: Avg. loss: 0.0013, Accuracy: 7925/10000 (79%)



plot confusion matrix

In [15]:
print_confusion_matrix(network, test_loader, num_classes=14)

confusion matrix: 
[[ 972    1    3    0    0    0    3    1    0    0    0    0    0]
 [   0 1120    4    2    0    1    2    5    0    0    0    1    0]
 [   1    0 1025    2    0    0    0    3    0    1    0    0    0]
 [   0    0    2  997    0    3    0    8    0    0    0    0    0]
 [   0    0    2    0  974    0    5    0    0    0    0    1    0]
 [   0    0    0   10    0  879    2    1    0    0    0    0    0]
 [   6    2    0    0    1    4  945    0    0    0    0    0    0]
 [   0    1   11    3    0    0    0 1013    0    0    0    0    0]
 [  19    7   46   48   22   79    8   14    0  600    0  129    2]
 [   5    3    1   11   80   48    0   32    0    0    1  828    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0]]


# Conclusion:
#### 1. Accuracy dropped to approximately 80% (79%) indicates lots of images belongs to '8' and '9' were not classified as '8' or '9' as before.
#### 2. Most images belongs to '8' and '9' were not classified as '0', '1', '2', '3', '4', '5', '6' or '7' which is exactly what we expected.
## Above 1. and 2. proved that the proposed 'NegativePenaltySparseCategoricalCrossentropy' loss function workd as expected